In [ ]:
import numpy as np
from mesh import solve

In [ ]:
def cpu_time_comparison(**kwargs):

    default_categories = [
        "no_prior",
        "with_prior_additive",
    ]
    default_nG = [1, 2, 3, 4]
    default_nx = 10 * 2 ** np.arange(5)

    categories = kwargs.get("categories", default_categories)
    all_nG = kwargs.get("all_nG", default_nG)
    all_nx = kwargs.get("all_nx", default_nx)
    n_runs = kwargs.get("n_runs", 5)

    cpu_times_mesh = np.zeros((n_runs, len(all_nx), len(all_nG), len(categories)))
    cpu_times_scheme = np.zeros((n_runs, len(all_nx), len(all_nG), len(categories)))

    for i_run in range(n_runs):
        print(f"run {i_run + 1}/{n_runs}")
        for i_nx, nx in enumerate(all_nx):
            for i_nG, nG in enumerate(all_nG):
                for i_c, category in enumerate(categories):
                    W, M = solve(category, nx, nG, True, 0.5, False)
                    cpu_times_mesh[i_run, i_nx, i_nG, i_c] = M.cpu_time_mesh
                    cpu_times_scheme[i_run, i_nx, i_nG, i_c] = M.cpu_time_scheme

    cpu_times_mesh = np.mean(np.sort(cpu_times_mesh, axis=0)[:3], axis=0)
    cpu_times_scheme = np.mean(np.sort(cpu_times_scheme, axis=0)[:3], axis=0)

    print()

    def to_ltx(f):
        float_str = f"{f:3.2e}"
        if "e" in float_str:
            base, exponent = float_str.split("e")
            return rf"${base} \cdot 10^{{{int(exponent)}}}$"
        else:
            return float_str

    for i_nG, nG in enumerate(all_nG):
        string = f"results for q = {nG - 1} \n \\toprule \n"
        string += " & \\multicolumn{3}{c}{basis $V_h$} & \\multicolumn{3}{c}{basis $V_h^+$} & \\\\ \n"
        string += "\\cmidrule(lr){2-4}\\cmidrule(lr){5-7}\n"
        string += "$K$"
        string += "".join([" & assembly & scheme & total" for category in categories])
        string += " & ratio \\\\ \n"
        string += "\\cmidrule(lr){1-1}\\cmidrule(lr){2-4}\\cmidrule(lr){5-7}\\cmidrule(lr){8-8}\n"

        for i_nx, nx in enumerate(all_nx):
            string += f"{nx:3d} "
            total = []
            for i_c, category in enumerate(categories):
                mesh = cpu_times_mesh[i_nx, i_nG, i_c]
                scheme = cpu_times_scheme[i_nx, i_nG, i_c]
                total.append(mesh + scheme)
                string += f"& {to_ltx(mesh)} & {to_ltx(scheme)} & {to_ltx(total[-1])}"
            string += f"& {total[1] / total[0]: 3.2f}"
            string += " \\\\ \n"
        string += "\\bottomrule \n"

        print(string)

In [ ]:
cpu_time_comparison(n_runs=100)

In [ ]:
import os

from model import Network

In [ ]:
epochs_and_times = []
n_simu = 10

for i_l, layer_sizes in enumerate(
    [
        [4, 16, 32, 16, 5],
        [4, 16, 32, 32, 16, 5],
        [4, 16, 32, 64, 32, 16, 5],
    ]
):
    for i in range(n_simu):

        print(f"Layer sizes: {i_l}, Simu: {i+1}/{n_simu}")

        try:
            os.remove("temp.pth")
        except FileNotFoundError:
            pass

        model = Network(file_name="temp.pth", layer_sizes=layer_sizes)
        model.train(n_epochs=25_000, n_collocation=5_000, n_data=0, plot=False)

        print(model.epochs_and_times)
        epochs_and_times.append(model.epochs_and_times)

print()
print("#############################################")
print(epochs_and_times)
print("#############################################")

with open(r"cpu_time_model.txt", "w") as file:
    file.write("\n".join(str(item) for item in epochs_and_times))

In [ ]:
model.plot_result()